In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.Container {width:85% !important;}
div.CodeMirror {font-family:NanumGothic; font-size:13pt; line-height : 150%;}
div.output_area pre {font-family:NanumGothic; font-size:13pt; line-height : 150%;}
div.output_wrapper pre {font-family:NanumGothic; font-size:13pt; line-height : 150%;}
div.input {font-family:NanumGothic; font-size:13pt; line-height : 150%;}
</style>
"""))

<b><font color = "red" size = "6">ch03. 연관분석</font></b>

# 1절. 연관분석 개요
- 데이터들 사이에서 자주 발생하는 속성을 찾고, 연관성이 있는지를 분석하는 방법
- 활용 분야 : 상품진열, 사기보험 적발, 카탈로그 디자인, 신상품 카테고리
```
    ▶ {조건} → {결과}
        ex) {소주, 콜라} → {와인} : 소주와 콜라를 사면 와인을 산다.
        ex) {와인} → {소주, 콜라} : 와인을 사면 소주와 콜라를 산다.
    
     위의 예시는 연관분석에서는 서로 다른 의미를 가진다. 이렇게 도출된 연관성 규칙들은 지지도, 향상도, 신뢰도를 통해
    얼마나 의미있는 규칙인지 평가한다.
    
    ▶ 연관분석 관련지표
        ① 지지도(support) - 조건결과항목 수 / 전체 수
            - 전체 거래 중 연관성 규칙을 구성하는 항목들이 포함된 거래의 비율
        ② 신뢰도 (confidence) - 조건결과항목 수 / 조건항목 수
            - 조건이 발생했을 때, 결과가 동시에 일어날 확률
        ③ 향상도 (lift) - 조건결과지지도 / ( (조건지지도) * (결과지지도) )
            - 우연히 발생한 규칙인지 아닌지 여부
            1 : 조건과 결과는 우연한 관게일 뿐 연관성 전혀 없음
            >1 (양의 상관관계) : 의미있는 연관성을 가진 규칙으로 해석, A를 구매했을 때 B를 구매할 확률이 더 크다.
            <1 (음의 상관관계) : A를 구매하면 B를 구매하지 않을 확률이 구매할 확률보다 크다.
        
    ▶ ex) {조건} → {결과}     지지도     신뢰도         향상도                               추론
           {주스} → {콜라}     2 / 5      2 / 2     0.4 / ( 0.4 * 1 ) = 1            연관성이 있다고 보지 않는다.
           {소주} → {맥주}     1 / 5      1 / 3     0.2 / ( 0.6 * 0.4 ) = 0.8333     약간의 연관성이 있다고 본다.
```

In [2]:
import os
os.getcwd()

'D:\\Gray_Bigdata\\src\\09_자연어처리'

In [3]:
%ls cf_basket.csv

 D 드라이브의 볼륨: 학생방
 볼륨 일련 번호: CA5B-F897

 D:\Gray_Bigdata\src\09_자연어처리 디렉터리

2021-08-19  오후 05:50               115 cf_basket.csv
               1개 파일                 115 바이트
               0개 디렉터리  197,241,745,408 바이트 남음


In [4]:
# 트랜잭션 데이터 가져오기
import csv
transaction = []
with open('cf_basket.csv', 'r', encoding = 'UTF8') as f:
    csvData = csv.reader(f)
    transaction = list(csvData)
#      = transaction.append(row)
transaction

[['소주', '콜라', '와인'],
 ['소주', '오렌지주스', '콜라'],
 ['콜라', '맥주', '와인'],
 ['소주', '콜라', '맥주'],
 ['오렌지주스', '와인']]

# 2. 연관분석
- pip install apyori

## 2.1 연관성 규칙 생성

In [5]:
from apyori import apriori
rules = apriori(transaction, 
                min_support = 0.2,
                min_confidence = 0.1)
rules = list(rules)

len(rules) # 규칙의 개수

18

In [6]:
list(rules[2])

[frozenset({'오렌지주스'}),
 0.4,
 [OrderedStatistic(items_base=frozenset(), items_add=frozenset({'오렌지주스'}), confidence=0.4, lift=1.0)]]

In [7]:
print('조건 → 결과\t지지  신뢰  향상')
for row in rules:
    support = row[1]
    orderedSt = row[2]
    for item in orderedSt:
        lhs = [ x for x in item[0] ]
        lhs = ','.join([ x for x in item[0] ])
        
        rhs = [ x for x in item[1] ]
        rhs = ','.join([ x for x in item[1] ])
        
        confidence = item[2]
        lift               = item[3]
        
        if lift > 1:
            print("{} → {}\t{}\t{}\t{}".format(lhs, rhs, round(support, 1), round(confidence, 1), round(lift, 1)))

조건 → 결과	지지  신뢰  향상
맥주 → 콜라	0.4	1.0	1.2
콜라 → 맥주	0.4	0.5	1.2
소주 → 콜라	0.6	1.0	1.2
콜라 → 소주	0.6	0.7	1.2
콜라 → 맥주,소주	0.2	0.2	1.2
맥주,소주 → 콜라	0.2	1.0	1.2
맥주 → 콜라,와인	0.2	0.5	1.2
콜라 → 맥주,와인	0.2	0.2	1.2
맥주,와인 → 콜라	0.2	1.0	1.2
콜라,와인 → 맥주	0.2	0.5	1.2
소주 → 콜라,오렌지주스	0.2	0.3	1.7
콜라 → 소주,오렌지주스	0.2	0.2	1.2
소주,오렌지주스 → 콜라	0.2	1.0	1.2
콜라,오렌지주스 → 소주	0.2	1.0	1.7
콜라 → 와인,소주	0.2	0.2	1.2
와인,소주 → 콜라	0.2	1.0	1.2


In [8]:
# 데이터 프레임으로 바꾸기
import pandas as pd
result_df = pd.DataFrame(None, columns = ['lhs', 'rhs', 'support', 'confidence', 'lift'])
index = 0

for row in rules:
    support = row[1]
    orderedSt = row[2]
    for item in orderedSt:
        lhs               = ','.join([ x for x in item[0] ])
        rhs              =  ','.join([ x for x in item[1] ])
        confidence = item[2]
        lift               = item[3]
        if lift > 1:
            result_df.loc[index] = [lhs, rhs, support, round(confidence, 2), round(lift, 2)]
            index += 1
            
result_df.sort_values(by = 'confidence')

,lhs,rhs,support,confidence,lift
4,콜라,"맥주,소주",0.2,0.25,1.25
7,콜라,"맥주,와인",0.2,0.25,1.25
11,콜라,"소주,오렌지주스",0.2,0.25,1.25
14,콜라,"와인,소주",0.2,0.25,1.25
10,소주,"콜라,오렌지주스",0.2,0.33,1.67
1,콜라,맥주,0.4,0.50,1.25
6,맥주,"콜라,와인",0.2,0.50,1.25
9,"콜라,와인",맥주,0.2,0.50,1.25
3,콜라,소주,0.6,0.75,1.25
0,맥주,콜라,0.4,1.00,1.25


# 3절. 뉴스기사 연관분석

## 3.1 뉴스 RSS를 이용해서 기사 검색 후 연관분석

In [9]:
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [12]:
# ▶ 1. url로부터 링크만 가져오기
rss_url = 'https://rss.joins.com/joins_money_list.xml'
money_response = urlopen(rss_url)

money_soup = BeautifulSoup(money_response, 'xml')
link_list = money_soup.select(' item > link ')

# link_list = [ link.text for link in link_list ]
link_list[ : 2]

[<link>https://news.joins.com/article/24132068?cloc=rss-news-economy</link>,
 <link>https://news.joins.com/article/24132026?cloc=rss-news-economy</link>]

In [20]:
# ▶ 2. link로 기사검색 후 명사만 추출해서 리스트에 담기
from konlpy.tag import Kkma
kkma = Kkma()
news = []
idx = 0

for link in link_list:
    news_response = requests.get(link.text)
    news_soup = BeautifulSoup(news_response.content, 'html.parser')
    elementTitle = news_soup.select_one(' h1.headline.mg ')
    
    if elementTitle:
        news_title = elementTitle.text
    else:
        news_title = ''
    
    news_article = news_soup.select_one(' div#article_body ').text
    news_content = news_title + news_article
    
    noun_list = list( filter( lambda word : len(word) > 1 & (word != '기자'), kkma.nouns(news_content) ) )
    news.append(noun_list)
    print(idx, '번 째 완료', end = '\t')
    idx += 1

0 번 째 완료	1 번 째 완료	2 번 째 완료	3 번 째 완료	4 번 째 완료	5 번 째 완료	6 번 째 완료	7 번 째 완료	8 번 째 완료	9 번 째 완료	10 번 째 완료	11 번 째 완료	12 번 째 완료	13 번 째 완료	14 번 째 완료	15 번 째 완료	16 번 째 완료	17 번 째 완료	18 번 째 완료	19 번 째 완료	20 번 째 완료	21 번 째 완료	22 번 째 완료	23 번 째 완료	24 번 째 완료	25 번 째 완료	26 번 째 완료	27 번 째 완료	28 번 째 완료	29 번 째 완료	

In [21]:
print( [ len(n) for n in news ] )

[92, 155, 271, 196, 191, 263, 252, 173, 247, 341, 224, 369, 128, 226, 262, 54, 161, 164, 39, 35, 216, 33, 38, 37, 45, 247, 159, 152, 237, 197]


In [22]:
# ▶ 3. 얻어진 list로 연관분석
from apyori import apriori
rules = apriori(news, 
                min_support = 0.2, 
                min_confidence = 0.2)

# 리스트로 안 바꾸면 generator 객체
rules = list(rules)

In [43]:
import pandas as pd
result_df = pd.DataFrame(None, 
                columns=['lhs','rhs','support','confidence','lift'])
index = 0
for row in rules:
    support = row[1]
    ordered_st = row[2]
    for item in ordered_st:
        lhs = ','.join(x for x in item[0])
        rhs = ','.join(x for x in item[1])
        confidence = item[2]
        lift      = item[3]
        if lift > 1:
            result_df.loc[index] = [lhs, rhs, support, confidence, lift]
            index += 1

In [44]:
result_df.loc[result_df.lhs.str.contains('올해|지난해')].sort_values(by=['lift','confidence'],
                                                               ascending=False).head(50)

,lhs,rhs,support,confidence,lift
2348,"관계자,지난해",업계,0.2,1.0,4.285714
4896,"관계자,지난해","19,업계",0.2,1.0,4.285714
4899,"19,지난해,관계자",업계,0.2,1.0,4.285714
5736,"관계자,지난해","19일,업계",0.2,1.0,4.285714
5739,"관계자,19일,지난해",업계,0.2,1.0,4.285714
6336,"관계자,지난해","기자,업계",0.2,1.0,4.285714
6341,"기자,관계자,지난해",업계,0.2,1.0,4.285714
6491,"상승,올해","평균,기준",0.2,1.0,4.285714
7659,"관계자,지난해","19,19일,업계",0.2,1.0,4.285714
7665,"19,지난해,관계자","19일,업계",0.2,1.0,4.285714


In [45]:
pd.options.display.max_rows = 100

In [46]:
result_df.loc[result_df.lhs.str.contains('올해|지난해') |
             result_df.rhs.str.contains('올해|지난해')].sort_values(by=['lift','confidence'],
                                                               ascending=False).head(75)

,lhs,rhs,support,confidence,lift
2348,"관계자,지난해",업계,0.2,1.000000,4.285714
4896,"관계자,지난해","19,업계",0.2,1.000000,4.285714
4899,"19,지난해,관계자",업계,0.2,1.000000,4.285714
5736,"관계자,지난해","19일,업계",0.2,1.000000,4.285714
5739,"관계자,19일,지난해",업계,0.2,1.000000,4.285714
6336,"관계자,지난해","기자,업계",0.2,1.000000,4.285714
6341,"기자,관계자,지난해",업계,0.2,1.000000,4.285714
6491,"상승,올해","평균,기준",0.2,1.000000,4.285714
7659,"관계자,지난해","19,19일,업계",0.2,1.000000,4.285714
7665,"19,지난해,관계자","19일,업계",0.2,1.000000,4.285714
